**CNN UNTUK GAMBAR**

---



6.2.1. Operasi Cross-Korelasi

Mari kita abaikan saluran untuk saat ini dan lihat cara kerjanya dengan data dua dimensi dan representasi tersembunyi. Inputnya adalah tensor dua dimensi dengan tinggi 3 dan lebar 3. Kami menandai bentuk tensor sebagai 3×3 atau ( 3 , 3 ). Tinggi dan lebar kernel keduanya 2. Bentuk jendela kernel (atau jendela konvolusi) diberikan oleh tinggi dan lebar kernel (ini dia 2×2 ).

In [8]:
!pip install d2l

     |████████████████████████████████| 82 kB 653 kB/s 
     |████████████████████████████████| 9.9 MB 56.3 MB/s 
     |████████████████████████████████| 11.6 MB 20.4 MB/s 
     |████████████████████████████████| 61 kB 9.9 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the p

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):  
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y


Kita dapat membangun tensor input X dan tensor kernel K dari Gambar 6.2.1 untuk memvalidasi output dari implementasi operasi korelasi silang dua dimensi di atas.

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

6.2.2. Lapisan Konvolusi

Lapisan convolutional mengkorelasikan silang input dan kernel dan menambahkan bias skalar untuk menghasilkan output. Dua parameter lapisan konvolusi adalah kernel dan bias skalar. Saat melatih model berdasarkan lapisan konvolusi, kami biasanya menginisialisasi kernel secara acak, seperti yang kami lakukan dengan lapisan yang terhubung penuh.

Kami sekarang siap untuk mengimplementasikan lapisan konvolusi dua dimensi berdasarkan fungsi corr2d yang didefinisikan di atas. Dalam fungsi konstruktor init, kami mendeklarasikan bobot dan bias sebagai dua parameter model. Fungsi propagasi maju memanggil fungsi corr2d dan menambahkan bias.

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

6.2.3. Deteksi Tepi Objek dalam Gambar

Mari kita luangkan waktu sejenak untuk mengurai aplikasi sederhana dari lapisan konvolusi: mendeteksi tepi suatu objek dalam gambar dengan menemukan lokasi perubahan piksel. Pertama, kami membuat "gambar" 6x8 piksel. Empat kolom tengah berwarna hitam dan sisanya berwarna putih

In [4]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

Selanjutnya, kita membangun kernel K dengan tinggi 1 dan lebar 2.

In [5]:
K = torch.tensor([[1.0, -1.0]])

untuk melakukan operasi korelasi silang dengan argumen X (input) dan K (kernel). Seperti yang terlihat, mendeteksi 1 untuk tepi dari putih ke hitam dan -1 untuk tepi dari hitam ke putih. Semua output lainnya mengambil nilai 0.

In [6]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

sekarang dapat menerapkan kernel ke gambar yang ditransposisikan

In [7]:
Z= X.t()
Z

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [8]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

6.2.4. Mempelajari Kernel

Sekarang mari kita lihat apakah kita dapat mempelajari kernel yang menghasilkan Y dari X dengan melihat pasangan input-output saja.

Kami pertama-tama membangun lapisan konvolusi dan menginisialisasi kernelnya sebagai tensor acak.
Selanjutnya, dalam setiap iterasi, kami akan menggunakan kesalahan kuadrat untuk membandingkan Y dengan output dari lapisan konvolusi.
Kami kemudian dapat menghitung gradien untuk memperbarui kernel.

In [10]:
# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2). For the sake of simplicity, we ignore the bias here

conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# The two-dimensional convolutional layer uses four-dimensional input and
# output in the format of (example, channel, height, width), where the batch
# size (number of examples in the batch) and the number of channels are both 1

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # Learning rate

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # Update the kernel
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

#Now we will take a look at the kernel tensor we learned.

conv2d.weight.data.reshape((1, 2))

epoch 2, loss 7.628
epoch 4, loss 1.752
epoch 6, loss 0.487
epoch 8, loss 0.161
epoch 10, loss 0.059


tensor([[ 1.0118, -0.9634]])